In [1]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os

import matplotlib.pyplot as plt

rng = 69

In [2]:
df = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\Kaggle\Insurance Premiums\Transformed.csv")
#df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Transformed_PCA - Test v1.csv")

In [3]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df, test_size=0.2, random_state=rng)


X_train = train_set.drop('Premium Amount', axis=1)
Y_train = train_set['Premium Amount'].copy()
Y_log_train = np.log1p(Y_train)

X_valid = train_set.drop('Premium Amount', axis=1)
Y_valid = train_set['Premium Amount'].copy()
Y_log_valid = np.log1p(Y_valid)

X_test = test_set.drop('Premium Amount', axis=1)
Y_test = test_set['Premium Amount'].copy()
Y_log_test = np.log1p(Y_test)

df_Final = df.drop('Premium Amount', axis=1)

In [4]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [5]:
# Fine Tune the hyperparameters of RandomForest model
# from sklearn.model_selection import GridSearchCV

# param_grid = [{'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},
#               {'bootstrap': [False], 'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},]

# rnd_clf_test = RandomForestClassifier()

# grid_search = GridSearchCV(rnd_clf_test, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

# grid_search.fit(nhl_prepared,nhl_labels)

# grid_search.best_params_

In [6]:
# cvres = grid_search.cv_results_
# results = []
# for mean_scores, params in zip(cvres["mean_test_score"], cvres["params"]):
#     results.append([np.sqrt(-mean_scores),params])

# results2 = sorted(results, key=lambda x: x[0])
# print(*results2, sep = '\n')

In [7]:
models = {}

#Linear Regression
from sklearn.linear_model import LinearRegression
models['Linear Regression'] = LinearRegression()

# # Support Vector Machines
# from sklearn.svm import SVR
# models['Support Vector Machines'] = SVR(kernel='rbf')

# # Decision Trees
# from sklearn.tree import DecisionTreeClassifier
# models['Decision Trees'] = DecisionTreeClassifier()

# # Random Forest
# from sklearn.ensemble import RandomForestClassifier
# models['Random Forest'] = RandomForestClassifier()

# # K-Nearest Neighbors
# from sklearn.neighbors import KNeighborsRegressor
# models['K-Nearest Neighbor'] = KNeighborsRegressor()

# # Bagging Decision Tree
# from sklearn.ensemble import BaggingClassifier
# from sklearn.tree import DecisionTreeClassifier
# models['Bagging Decision Tree'] =  BaggingClassifier(DecisionTreeClassifier())

# # Extra Random Forest
# from sklearn.ensemble import ExtraTreesClassifier
# models['Extra Random Forest'] = ExtraTreesClassifier()

# # Gradient Boosting
# from sklearn.ensemble import GradientBoostingRegressor
# models['Gradient Boosting'] = GradientBoostingRegressor()

# XG Boost
from xgboost import XGBRegressor
models['XG Boost'] = XGBRegressor()

# # ADA Boost
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.tree import DecisionTreeClassifier
# models['ADA Boost'] = AdaBoostClassifier(DecisionTreeClassifier())

# Baysian Regression
from sklearn.linear_model import BayesianRidge
models['Bayesian Regression'] = BayesianRidge()

In [8]:
from sklearn.metrics import mean_squared_log_error
import numpy as np

# Initialize a dictionary to store MSE for each model
rmsle_scores = {}

for key in models.keys():
    
    if key == "Linear Regression":
        # Fit the model
        models[key].fit(X_train_prepared, Y_log_train) 
        # Make predictions
        Y_log_pred = models[key].predict(X_test_prepared)  
        # Inverse transform predictions to the original scale (exponentiate)
        Y_pred_original = np.expm1(Y_log_pred)  # Inverse of log transformation  
        # Calculate MSE for this model
        rmsle_scores[key] = np.sqrt(mean_squared_log_error(Y_test, Y_pred_original)) 
    else:
        models[key].fit(X_train_prepared, Y_train)
        Y_pred = models[key].predict(X_test_prepared)
        # Shift the data to ensure non-negative values
        shift_value = abs(min(Y_test.min(), Y_pred.min())) + 1  # Adding 1 ensures all values are positive
        
        # Apply the transformation
        Y_pred_shifted = Y_pred + shift_value
        rmsle_scores[key] = np.sqrt(mean_squared_log_error(Y_test, Y_pred_shifted)) 

In [9]:
import pandas as pd

df_model = pd.DataFrame(index=models.keys(), columns=['RMSLE'])
df_model['RMSLE'] = rmsle_scores.values()

df_model

,RMSLE
Linear Regression,1.254449
XG Boost,1.373155
Bayesian Regression,1.369023
